# Time Series Forecasting: Electricity, Traffic, Exchange Rate
This notebook loads `.txt` datasets and trains Linear, DLinear and NLinear models using PyTorch.

In [1]:
# Imports
import pandas as pd
import numpy as np
import gzip
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer


device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# Load plain .txt file
def load_data(filepath):
    return pd.read_csv(filepath, header=None)

# Load all datasets from .txt
datasets = {
    'electricity': load_data('electricity.txt'),
    'traffic': load_data('traffic.txt'),
    'exchange_rate': load_data('exchange_rate.txt')
}


In [3]:
for name in datasets:
    df = pd.DataFrame(datasets[name])
    datasets[name] = ((df - df.min()) / (df.max() - df.min())).values.astype(np.float32)


In [4]:
def create_sequences(data, input_len, output_len):
    X, y = [], []
    for i in range(len(data) - input_len - output_len):
        X.append(data[i:i+input_len])
        y.append(data[i+input_len:i+input_len+output_len])
    X = np.array(X)
    y = np.array(y)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


In [5]:
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# 1. Linear
class LinearModel(nn.Module):
    def __init__(self, input_len, output_len, features):
        super().__init__()
        self.linear = nn.Linear(input_len, output_len)
    def forward(self, x):
        return self.linear(x.transpose(1, 2)).transpose(1, 2)

# 2. DLinear
class DLinear(nn.Module):
    def __init__(self, input_len, output_len, features):
        super().__init__()
        self.decomp = nn.Linear(input_len, input_len)
        self.proj = nn.Linear(input_len, output_len)
    def forward(self, x):
        x = x.transpose(1, 2)
        seasonal = self.decomp(x)
        out = self.proj(seasonal)
        return out.transpose(1, 2)

# 3. NLinear
class NLinear(nn.Module):
    def __init__(self, input_len, output_len, features):
        super().__init__()
        self.norm = nn.LayerNorm(features)
        self.linear = nn.Linear(input_len, output_len)
    def forward(self, x):
        x = self.norm(x)
        return self.linear(x.transpose(1, 2)).transpose(1, 2)

# 4. Transformer (corregido)
class TransformerModel(nn.Module):
    def __init__(self, input_len, output_len, features, d_model=512, nhead=8, num_layers=2):
        super().__init__()
        self.input_proj = nn.Linear(features, d_model)
        encoder_layer = TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer = TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_proj = nn.Linear(d_model, features)
        self.output_len = output_len

    def forward(self, x):
        # x: [B, L, F]
        x = self.input_proj(x)        # -> [B, L, d_model]
        x = x.transpose(0, 1)         # -> [L, B, d_model]
        x = self.transformer(x)       # -> [L, B, d_model]
        x = x.transpose(0, 1)         # -> [B, L, d_model]
        x = self.output_proj(x)       # -> [B, L, F]

        # Ajustamos para que la salida tenga length = output_len (cortamos o interpolamos)
        if x.shape[1] > self.output_len:
            x = x[:, -self.output_len:, :]  # Cortamos las últimas output_len
        elif x.shape[1] < self.output_len:
            # Upsample temporalmente si es necesario
            x = nn.functional.interpolate(x.transpose(1, 2), size=self.output_len, mode='linear', align_corners=False)
            x = x.transpose(1, 2)

        return x  # [B, output_len, features]


# 5. Autoformer (ya corregido)
class Autoformer(nn.Module):
    def __init__(self, input_len, output_len, features):
        super().__init__()
        self.encoder = nn.Linear(features, features)
        self.decomp = nn.Linear(features, features)
        self.output_proj = nn.Linear(input_len, output_len)
    def forward(self, x):
        trend = self.encoder(x)
        seasonal = x - trend
        seasonal_decomp = self.decomp(seasonal)
        seasonal_decomp = seasonal_decomp.transpose(1, 2)
        out = self.output_proj(seasonal_decomp)
        return out.transpose(1, 2)
class Informer(nn.Module):
    def __init__(self, input_len, output_len, features):
        super().__init__()
        self.input_proj = nn.Linear(features, features)
        self.attn = nn.MultiheadAttention(embed_dim=features, num_heads=1, batch_first=True)
        self.output_proj = nn.Linear(input_len, output_len)  # Proyecta a output_len

    def forward(self, x):
        # x: [B, L, F]
        x = self.input_proj(x)  # [B, L, F]
        attn_output, _ = self.attn(x, x, x)  # [B, L, F]
        attn_output = attn_output.transpose(1, 2)  # [B, F, L]
        out = self.output_proj(attn_output)  # [B, F, output_len]
        return out.transpose(1, 2)  # [B, output_len, F]



# 7. Reformer (corregido)
class Reformer(nn.Module):
    def __init__(self, input_len, output_len, features):
        super().__init__()
        self.linear = nn.Linear(input_len, output_len)
    def forward(self, x):
        return self.linear(x.transpose(1, 2)).transpose(1, 2)

# 8. FEDformer (corregido)
class FEDformer(nn.Module):
    def __init__(self, input_len, output_len, features):
        super().__init__()
        self.filter = nn.Linear(input_len, input_len)
        self.output_proj = nn.Linear(input_len, output_len)
    def forward(self, x):
        freq = torch.fft.rfft(x, dim=1)
        filtered = torch.fft.irfft(freq, n=x.size(1), dim=1)
        filtered = filtered.transpose(1, 2)
        out = self.output_proj(self.filter(filtered))
        return out.transpose(1, 2)


In [6]:
def train_model(model, X_train, y_train, epochs=10, lr=1e-3, batch_size=64):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    dataset = torch.utils.data.TensorDataset(X_train, y_train)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    print(f"\n--- Training {model.__class__.__name__} ---")
    print(f"Input shape: {X_train.shape}, Target shape: {y_train.shape}")
    print(f"Device: {device}")

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        for batch_idx, (x_batch, y_batch) in enumerate(loader):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            output = model(x_batch)

            if output.shape != y_batch.shape:
                print(f"[!] Shape mismatch at batch {batch_idx}: output {output.shape}, target {y_batch.shape}")
                continue

            loss = loss_fn(output, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.4f}")

    print(f"Training complete for {model.__class__.__name__} ✅\n")
    return model


In [7]:
# Evaluation with debug info
def evaluate_model(model, X_test, y_test):
    model.eval()
    print(f"\n--- Evaluating {model.__class__.__name__} ---")
    print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
    
    with torch.no_grad():
        output = model(X_test.to(device))
        pred = output.cpu().numpy()
        true = y_test.cpu().numpy()
    
    print(f"Prediction shape: {pred.shape}, Ground truth shape: {true.shape}")
    
    # Show first few predictions vs true values
    print("\nFirst 5 predictions vs. true values (flattened):")
    for i in range(5):
        print(f"Pred: {pred[i].flatten()[:5]} | True: {true[i].flatten()[:5]}")
    
    mse = mean_squared_error(true.flatten(), pred.flatten())
    mae = mean_absolute_error(true.flatten(), pred.flatten())
    rse = np.sqrt(mse) / np.std(true.flatten())
    corr = np.corrcoef(true.flatten(), pred.flatten())[0, 1]
    
    print(f"\nMetrics:")
    print(f" - MSE:  {mse:.6f}")
    print(f" - MAE:  {mae:.6f}")
    print(f" - RSE:  {rse:.6f}")
    print(f" - Corr: {corr:.6f}")
    
    return mse, mae, rse, corr


In [8]:
# Run all models on all datasets with debug info
input_len, output_len = 96, 96
results = []

# Lista completa de modelos
model_classes = [LinearModel, DLinear, NLinear, TransformerModel, Autoformer, Informer, Reformer, FEDformer]

for name, data in datasets.items():
    print(f"\n=== Dataset: {name} ===")
    print(f"Original data shape: {data.shape}")

    X, y = create_sequences(data, input_len, output_len)
    print(f"Created sequences -> X: {X.shape}, y: {y.shape}")

    X_train, y_train = X[:500], y[:500]
    X_test, y_test = X[-100:], y[-100:]
    print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")

    for Model in model_classes:
        print(f"\n--- Training model: {Model.__name__} ---")
        model = Model(input_len, output_len, data.shape[1])
        model = train_model(model, X_train, y_train, epochs=10)
        print(f"Finished training {Model.__name__}, now evaluating...")

        mse, mae, rse, corr = evaluate_model(model, X_test, y_test)
        results.append([name, Model.__name__, mse, mae, rse, corr])



=== Dataset: electricity ===
Original data shape: (26304, 321)
Created sequences -> X: torch.Size([26112, 96, 321]), y: torch.Size([26112, 96, 321])
Train size: 500, Test size: 100

--- Training model: LinearModel ---

--- Training LinearModel ---
Input shape: torch.Size([500, 96, 321]), Target shape: torch.Size([500, 96, 321])
Device: cpu
Epoch 1/10 | Loss: 1.5277
Epoch 2/10 | Loss: 0.4801
Epoch 3/10 | Loss: 0.3108
Epoch 4/10 | Loss: 0.2766
Epoch 5/10 | Loss: 0.2386
Epoch 6/10 | Loss: 0.2036
Epoch 7/10 | Loss: 0.1837
Epoch 8/10 | Loss: 0.1688
Epoch 9/10 | Loss: 0.1557
Epoch 10/10 | Loss: 0.1448
Training complete for LinearModel ✅

Finished training LinearModel, now evaluating...

--- Evaluating LinearModel ---
X_test shape: torch.Size([100, 96, 321]), y_test shape: torch.Size([100, 96, 321])
Prediction shape: (100, 96, 321), Ground truth shape: (100, 96, 321)

First 5 predictions vs. true values (flattened):
Pred: [0.13419798 0.3476071  0.07159282 0.7333722  0.7076262 ] | True: [0.08


Metrics:
 - MSE:  0.031938
 - MAE:  0.145906
 - RSE:  0.932572
 - Corr: 0.402984

--- Training model: Reformer ---

--- Training Reformer ---
Input shape: torch.Size([500, 96, 321]), Target shape: torch.Size([500, 96, 321])
Device: cpu
Epoch 1/10 | Loss: 1.5654
Epoch 2/10 | Loss: 0.5225
Epoch 3/10 | Loss: 0.3134
Epoch 4/10 | Loss: 0.2606
Epoch 5/10 | Loss: 0.2221
Epoch 6/10 | Loss: 0.1928
Epoch 7/10 | Loss: 0.1742
Epoch 8/10 | Loss: 0.1602
Epoch 9/10 | Loss: 0.1484
Epoch 10/10 | Loss: 0.1390
Training complete for Reformer ✅

Finished training Reformer, now evaluating...

--- Evaluating Reformer ---
X_test shape: torch.Size([100, 96, 321]), y_test shape: torch.Size([100, 96, 321])
Prediction shape: (100, 96, 321), Ground truth shape: (100, 96, 321)

First 5 predictions vs. true values (flattened):
Pred: [0.0716575  0.2972717  0.01694185 0.4780666  0.53574705] | True: [0.08571429 0.35135135 0.01331115 0.85128206 0.68007314]
Pred: [0.05169513 0.27686945 0.01764398 0.475769   0.55101144] 

Prediction shape: (100, 96, 862), Ground truth shape: (100, 96, 862)

First 5 predictions vs. true values (flattened):
Pred: [0.11253129 0.12379447 0.1956426  0.11379723 0.02615889] | True: [0.067411   0.06266667 0.13068317 0.09116636 0.02482182]
Pred: [0.10162337 0.10808837 0.16405211 0.10127406 0.01791494] | True: [0.05940594 0.05733333 0.11283028 0.08712351 0.02482182]
Pred: [0.0878823  0.0934301  0.12862702 0.08277534 0.00536277] | True: [0.0471877  0.05177778 0.1061652  0.08954922 0.02113541]
Pred: [ 0.06533606  0.07562378  0.08191141  0.06667927 -0.00877331] | True: [0.01558879 0.03711111 0.07736253 0.0667071  0.01474564]
Pred: [ 0.0515254   0.07441115  0.04380275  0.0492508  -0.01331977] | True: [0.01200758 0.02488889 0.05236848 0.04386497 0.0093389 ]

Metrics:
 - MSE:  0.007510
 - MAE:  0.058197
 - RSE:  0.889132
 - Corr: 0.575700

--- Training model: Autoformer ---

--- Training Autoformer ---
Input shape: torch.Size([500, 96, 862]), Target shape: torch.Size([500, 96, 862])
De

Epoch 1/10 | Loss: 1.0190
Epoch 2/10 | Loss: 0.1977
Epoch 3/10 | Loss: 0.1053
Epoch 4/10 | Loss: 0.0804
Epoch 5/10 | Loss: 0.0626
Epoch 6/10 | Loss: 0.0559
Epoch 7/10 | Loss: 0.0523
Epoch 8/10 | Loss: 0.0496
Epoch 9/10 | Loss: 0.0480
Epoch 10/10 | Loss: 0.0464
Training complete for DLinear ✅

Finished training DLinear, now evaluating...

--- Evaluating DLinear ---
X_test shape: torch.Size([100, 96, 8]), y_test shape: torch.Size([100, 96, 8])
Prediction shape: (100, 96, 8), Ground truth shape: (100, 96, 8)

First 5 predictions vs. true values (flattened):
Pred: [0.42544806 0.27832043 0.33572254 0.5814743  0.36068088] | True: [0.41392905 0.2498323  0.33197728 0.59765685 0.33024514]
Pred: [0.42382136 0.27657074 0.3335698  0.58244306 0.36049426] | True: [0.42368132 0.28414446 0.3417586  0.5950704  0.33227372]
Pred: [0.42794484 0.28299025 0.3408599  0.5833046  0.36062866] | True: [0.42507336 0.2849122  0.3430463  0.59630364 0.32990316]
Pred: [0.42600024 0.2812826  0.33848634 0.5833302  0.36

Epoch 3/10 | Loss: 0.0953
Epoch 4/10 | Loss: 0.0770
Epoch 5/10 | Loss: 0.0586
Epoch 6/10 | Loss: 0.0539
Epoch 7/10 | Loss: 0.0501
Epoch 8/10 | Loss: 0.0477
Epoch 9/10 | Loss: 0.0461
Epoch 10/10 | Loss: 0.0447
Training complete for FEDformer ✅

Finished training FEDformer, now evaluating...

--- Evaluating FEDformer ---
X_test shape: torch.Size([100, 96, 8]), y_test shape: torch.Size([100, 96, 8])
Prediction shape: (100, 96, 8), Ground truth shape: (100, 96, 8)

First 5 predictions vs. true values (flattened):
Pred: [0.41382623 0.27484217 0.3467372  0.5553512  0.34300166] | True: [0.41392905 0.2498323  0.33197728 0.59765685 0.33024514]
Pred: [0.41260695 0.27304646 0.3427716  0.5537696  0.34267244] | True: [0.42368132 0.28414446 0.3417586  0.5950704  0.33227372]
Pred: [0.4118673  0.27340555 0.34030244 0.5514345  0.34179413] | True: [0.42507336 0.2849122  0.3430463  0.59630364 0.32990316]
Pred: [0.41047004 0.27626827 0.3408228  0.5534482  0.34133342] | True: [0.429545   0.28207418 0.33937

In [9]:
# Show results
df_results = pd.DataFrame(results, columns=['Dataset', 'Model', 'MSE', 'MAE', 'RSE', 'Corr'])
df_results

,Dataset,Model,MSE,MAE,RSE,Corr
0,electricity,LinearModel,0.020631,0.107505,0.749525,0.673790
1,electricity,DLinear,0.015214,0.088448,0.643646,0.766965
2,electricity,NLinear,0.145017,0.328001,1.987171,0.497542
3,electricity,TransformerModel,0.035068,0.148349,0.977189,0.441792
4,electricity,Autoformer,0.031649,0.138839,0.928334,0.476035
5,electricity,Informer,0.031938,0.145906,0.932572,0.402984
6,electricity,Reformer,0.020012,0.105482,0.738192,0.683831
7,electricity,FEDformer,0.015165,0.089249,0.642613,0.767015
8,traffic,LinearModel,0.005014,0.046658,0.726512,0.711288
9,traffic,DLinear,0.004102,0.040830,0.657097,0.762153
